In [ ]:
import mlflow
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 12
plt.rcParams['figure.autolayout'] = True
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), nrows=2)
ax[0].set_title("Running Loss per Curriculum")
ax[0].set_yscale("log")
ax[0].set_yticks([0.7, 1])
ax[0].get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
client = mlflow.tracking.MlflowClient()

for run in mlflow.search_runs('0')[:3].run_id:
    metrics = client.get_metric_history(run, 'eval_loss')
    loss = [x.value for x in metrics]
    step = [x.step for x in metrics]
    minimal = step[np.argmin(loss)]
    name = client.get_run(run).data.params["output_dir"].lstrip("models/")
    ax[0].plot(step, loss, label=name)
    p = ax[1].plot(step, loss, label=name)
    ax[1].axvline(minimal, color=p[0].get_color(), linestyle="--", linewidth=1)
    ax[1].set_xlim(3000, 6000)
    ax[1].set_ylim(0.62, 0.67)
ax[1].set_xlabel("Steps")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Loss")
ax[0].legend()
fig.savefig("loss-medium.pdf", dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), nrows=2)
ax[0].set_title("Running Loss per Curriculum")
ax[0].set_yscale("log")
ax[0].set_yticks([0.7, 1, 2, 3])
ax[0].get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
client = mlflow.tracking.MlflowClient()

for run in mlflow.search_runs('0')[3:].run_id:
    metrics = client.get_metric_history(run, 'eval_loss')
    loss = [x.value for x in metrics]
    step = [x.step for x in metrics]
    minimal = step[np.argmin(loss)]
    name = client.get_run(run).data.params["output_dir"].lstrip("models/")
    ax[0].plot(step, loss, label=name)
    p = ax[1].plot(step, loss, label=name)
    ax[1].axvline(minimal, color=p[0].get_color(), linestyle="--", linewidth=1)
    ax[1].set_xlim(4000, 13000)
    ax[1].set_ylim(0.64, 0.675)
ax[1].set_xlabel("Steps")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Loss")
ax[0].legend()
fig.savefig("loss-small.pdf", dpi=300)

In [ ]:
!pip install ecco

In [1]:
import ecco

In [2]:
model_config = {
    'embedding': "transformer.wte.weight",
    'type': 'causal',
    'activations': ['mlp\.c_proj'], #This is a regex
    'token_prefix': 'Ġ',
    'partial_token_prefix': ''
}

explain = ecco.from_pretrained("models/gpt2-medium/ta_encode", activations=True, model_config=model_config)

In [ ]:
output = explain.generate("<sos_u>we are a family of 3 planning staying a week in rio de janeiro. "
                 "any suggestions for area , nice restaurant etc ?<eos_u><sos_b>",
                generate=1, attribution=['ig'])

<IPython.core.display.Javascript object>

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
output.primary_attributions(attr_method='ig')

In [ ]:
output.primary_attributions(attr_method='ig', style="detailed")

In [ ]:
output.layer_predictions(position=30, layer=11)